# Sobol Sensitivity Analysis

#### Notes
This program will be used to conduct Sobol SA on 12 preselected variables from Daniel's work

#### Workflow
1) Define parameters being considered, and parametric uncertainty associated with them
2) Use Sobol sequence to sample the parameter space, generating a matrix of p(n+2) x p
3) Convert values back to parameter values
4) Write each row of parameter values into an idf
5) Run the idfs to determine the output results
6) Calculate sobol first order and total order indices based on these results



In [34]:
# Importing libraries
from pathlib import Path
import time
from eppy.modeleditor import IDF
import pandas as pd
import numpy as np
import shutil
import os
from mpi4py import MPI
from scipy.stats import norm, qmc
import SALib
from SALib.sample import sobol


In [156]:
# Configurations

idd_file_path = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/Energy+.idd" # Change to your IDD file path
skeleton_idf_path = Path("/jumbo/keller-lab/Jeremy_Wang/eplus_sa/data/SingleFamilyHouse_TwoSpeed_CutoutTemperature.idf") # Change to your skeleton IDF path
work_dir = Path("/jumbo/keller-lab/Jeremy_Wang/eplus_sa/scripts/main") # Change to your working directory
# base_output_idf_dir = work_dir / "randomized_idfs"
# param_dir = work_dir / "params"

# Set IDD and working dir
IDF.setiddname(idd_file_path)
os.chdir(work_dir)
# base_output_idf_dir.mkdir(exist_ok=True)
# param_dir.mkdir(exist_ok=True)

### Sobol sequence random sample generation

In [157]:
# Unlike LHS, Sobol Sequence takes in a range of values then generates random samples from there

# We can start with the same baseline mean and std deviation values as Daniel

# standard deviation for each paramter is 5% of its original value
sd_frac = 0.05
# Means for each parameter
means = {
    'heating_sp': 22.0,
    'cooling_sp': 26.6,
    'people_per_area': 3.0,
    'infil_flow_rate': 0.01,
    'watts_equip': 500,
    'watts_lights': 1000,
    'heating_COP': 4.0,
    'fan_efficiency': 0.7,
    'pressure_rise': 400.0,
    'solar_transmittance': 0.837,
    'burner_eff': 0.8,
    'vent_flow_rate': 0.131944
}

# additional variable for gap between heating and cooling
gap_mean = means['cooling_sp'] - means['heating_sp']
gap_sd = np.sqrt((sd_frac*means['heating_sp'])**2 + (sd_frac*means['cooling_sp'])**2)
min_gap = 4 # establish minimum 4 degrees between heating and cooling setpoint

# we can define the range as 99.7% interval around the mean, equivalent to 3 STD
k = 3

# generate the bounds for each of the parameters
parameter_std = {}
parameter_names = []

for name, mean in means.items():
    # add parameters to a list
    parameter_names.append(name)

    # adding std into dictionary
    parameter_std[name] = mean*sd_frac

# appending on gap
parameter_names.append('gap')
parameter_std['gap'] = gap_sd

# manually change burner efficiency
parameter_std['burner_eff'] = sd_frac

# Creating parameter bounds
parameter_bounds = []
for name, mean in means.items():
    lbound = mean - k*parameter_std[name]
    ubound = mean + k*parameter_std[name]
    parameter_bounds.append([lbound,ubound])

# adding bounds for gap
parameter_bounds.append([min_gap,(gap_mean + k*gap_sd)])

In [158]:

# creating problem dictionary for sobol sampling
# requires first initially excluding the cooling setpoint because it is dependent on the heating setpoint
problem = {}

names_for_problem = [n for n in parameter_names if n != 'cooling_sp']
bounds_for_problem = [b for n,b in zip(parameter_names, parameter_bounds) if n != 'cooling_sp']

# this is the formatting needed for SA Lib to generate Sobol sequence from
problem = {
    'num_vars': len(names_for_problem),
    'names': names_for_problem,
    'bounds': bounds_for_problem
}

# conduct sobol sequence sampling
N = 1024 # baseline number of samples
param_values = sobol.sample(problem, N, calc_second_order=False) # array with dimensions [N*(P+2), P]

In [159]:
## reintegrating the cooling setpoint into the generated sample

# Find the column index of heating_sp
heating_idx = problem['names'].index('heating_sp')
gap_idx = problem['names'].index('gap')

# Extract heating_sp samples from param_values
heating_samples = param_values[:, heating_idx]
gap_samples = param_values[:, gap_idx]
cooling_samples = heating_samples + gap_samples # calculate cooling samples

#  update parameter values to reinsert cooling setpoint
param_values = np.insert(param_values, heating_idx+1, cooling_samples, axis=1) # inserting it in correct index
param_values = np.delete(param_values, -1, axis=1) # deleting the gap column of values
parameter_names.pop() # delete gap from list of parameters
parameter_bounds.pop()

# convert param_values into a list of dictionaries, where the keys correspond to the input parameters
samples = []
for i in range(param_values.shape[0]):
    sample_dict = {}
    for j in range(param_values.shape[1]):
        sample_dict[parameter_names[j]] = param_values[i,j]
    samples.append(sample_dict)


In [172]:
# validating samples 
invalid_samples = []
for i, dict in enumerate(samples):
    # making sure heating point is below cooling point
    if  dict['heating_sp'] > dict['cooling_sp']:
        invalid_samples.append(i)
        continue 
    # making sure samples are within the correct bounds
    for name, value in sample_dict.items():
        idx = parameter_names.index(name)  
        lower, upper = parameter_bounds[idx]
        if value < lower or value > upper:
            invalid_samples.append(i)
            break  # stop checking this sample

if not invalid_samples:
    print("No invalid samples")
else:
    print("Invalid samples:", invalid_samples)


No invalid samples


In [155]:

parameter_bounds[0][0]

18.7